In [1]:
# .py file with helper functions

In [308]:
import numpy as np
import pandas as pd
import re
import os
import wikipedia
import nltk.data
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.stem import WordNetLemmatizer
from itertools import chain
from difflib import get_close_matches as gcm
import string
from imdb import IMDb
import wikia
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [47]:
#function to extract list of wiki page titles
def titles_list(filepath):
    movies = pd.read_excel(open(filepath,'rb'))
    titles_list = movies['wikipedia page title']
    return titles_list

In [48]:
# function to extract imdb movie ids
def imdb_id_list(filepath):
    movies = pd.read_excel(open(filepath,'rb'))
    id_list = movies['imdb id']
    return id_list

In [49]:
#function to pull the text of plot section from a movie wiki page
def plot_puller_wiki(movie):
    movie_plot = wikipedia.WikipediaPage(title = movie).section('Plot')
    movie_plot = movie_plot.replace('\n','').replace("\'","").replace("\\","").lower()
    return movie_plot

In [50]:
def plot_puller_imdb(imdb_movie_id):
    ia = IMDb()
    movie = ia.get_movie(imdb_movie_id)
    movie_plot = str(movie['synopsis'])
    movie_plot = movie_plot.replace('\n','').replace("\'","").replace("\\","").lower()
    return movie_plot

In [51]:
# function to aggregate plots of all movies to be inclued in the corpus
def plot_aggregator_wiki(titles_list):
    plot_agg = ''
    for i, movie in enumerate(titles_list):
        movie_plot = plot_puller_wiki(movie)
        plot_agg += movie_plot
    return plot_agg

In [52]:
def plot_aggregator_imdb(imdb_movie_id_list):
    plot_agg = ''
    for i, movie_id in enumerate(imdb_movie_id_list):
        movie_plot = plot_puller_imdb(movie_id[2:])
        plot_agg += movie_plot
    return plot_agg

In [53]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [54]:
# function to lemmatize words according to corresponding part of speech
# will try to include different lemmatizers i.espacey, stanford, textblob
def lemma(word):
    if get_wordnet_pos(word) == 'r':
        try:
            possible_adj = []
            for ss in wordnet.synsets(word):
              for lemmas in ss.lemmas(): # all possible lemmas
                  for ps in lemmas.pertainyms(): # all possible pertainyms
                      possible_adj.append(ps.name())
            word = gcm(word,possible_adj)[0]
        except:
            word = word
    lemmatizer = WordNetLemmatizer()
    pos = get_wordnet_pos(word)
    lemmatized_word = lemmatizer.lemmatize(word, get_wordnet_pos(word))
    return lemmatized_word

In [142]:
# function to clean up and tokenize the raw text of the corpus
def preprocess_corpus_text(raw_string,lemmatize=True):
    raw_string = raw_string.lower()
    transtable = str.maketrans('', '', string.punctuation)
    raw_string = re.sub(r'(?<=[.,])(?=[^\s])', r' ', raw_string)
    stop_words=set(stopwords.words('english'))
    wordnet_lemmatizer = WordNetLemmatizer()
    sentence_tokens = sent_tokenize(raw_string)
    word_tokens  = []
    for sentence in sentence_tokens:
        clean_sentence = sentence.translate(transtable)
        tok = word_tokenize(clean_sentence)
        word_tokens.append(tok)
    final_tokens = []
    for sentence in word_tokens:
        ntk = [w for w in sentence if not w in stop_words]
        final_tokens.append(ntk)
    if lemmatize:
        lemmatized_tokens = []
        for i in range(len(final_tokens)):
            wordtoks = []
            for word in final_tokens[i]:
                wordlemma = lemma(word)
                wordtoks.append(wordlemma)
            lemmatized_tokens.append(wordtoks)
        while [] in lemmatized_tokens:
            lemmatized_tokens.remove([])
        return lemmatized_tokens
    else:
        while [] in final_tokens:
            final_tokens.remove([])
        return final_tokens

In [560]:
# function to pull plot synopsis from individual marvel wikia pages
def comic_plot(comic_vol_issue, wiki = 'marvel'):
    try:
        #summary = wikia.summary(wiki, comic_vol_issue)
        #summary = re.sub(r'(?<=[.,])(?=[^\s])', r' ', summary)
        full_page = wikia.page(wiki, comic_vol_issue)
        pg = full_page.content
        pg = re.sub(r'(?<=[.,])(?=[^\s])', r' ', pg)
        #key = summary[:key_length]
        #start_index = pg.find(key)
        #plot = pg[start_index:]
        plot = pg.lower()
        rm_list = ['featured characters:','supporting characters:','antagonists:', 'races and species:',
                   'other characters:','locations:','items:','vehicles:','\n','\xa0']
        plot = plot.replace('‘','\'').replace('...','.').replace('•','.').replace("\'","").replace('s.h.i.e.l.d.','SHIELD ').replace('’','\'').replace('s. h. i. e. l. d. ','SHIELD ')
        for item in rm_list:
            plot = plot.replace(item,'')
        if plot == '':
            return None
        else:
            return plot
    except:
        return None

In [376]:
# function to find comic book titles in each 'all comics' page
def comic_titles_finder(my_url):
    uClient = uReq(my_url)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html,'html.parser')
    next_page_tag = page_soup.findAll('a',{'class':'category-page__pagination-next wds-button wds-is-secondary'})
    if next_page_tag:
        next_page_link = next_page_tag[0]['href']
    else:
        next_page_link = None
    comic_titles = page_soup.findAll('a',{'class':'category-page__member-link'})
    comic_vol_issue_list = []
    for tag in comic_titles:
        name = tag['title']
        if 'Category:' not in name:
            comic_vol_issue_list.append(name)
    return comic_vol_issue_list, next_page_link

In [387]:
# function to go through all 'all comics' pages and create csv containing full names of all comic books
def make_comic_titles_list(page_url,target_file):
    filename = target_file + '.csv'
    f = open(filename,'w')
    headers = 'comic_title\n'
    f.write(headers)
    while page_url:
        cl,page_url = comic_titles_finder(page_url)
        for title in cl:
            f.write(title.replace(',','')+'\n')
    f.close()

In [ ]:
'''RUN ONLY ONCE'''
#make_comic_titles_list(first_page_url,'comic_titles')

In [468]:
# function to aggregate plots of all comic books
# manually delete target_file if it already exists
def comic_plot_agg(titles_list,target_file):
    filename = target_file + '.txt'
    for i,title in enumerate(titles_list):
        plot = comic_plot(title)
        if plot:
            with open(filename, 'a+') as f:
                f.write(plot)

In [552]:
cl = pd.read_csv('comic_titles.csv',header=0,encoding = 'unicode_escape')['comic_title']

In [553]:
s = cl[:1000]#['comic_title']

In [554]:
s

0                             All-New Wolverine Vol 1 1
1                           Extraordinary X-Men Vol 1 2
2                                    Hail Hydra Vol 1 4
3                                         Thors Vol 1 4
4                                X-Men Classic Vol 1 57
5                                         Hulk Vol 3 10
6                     Essential Series Vol 1 Avengers 8
7                                         Hulk Vol 3 11
8      X-Men Archives Featuring Captain Britain Vol 1 1
9                      Essential Series Vol 1 Dazzler 1
10                     1602 Witch Hunter Angela Vol 1 4
11                            Age of Apocalypse Vol 2 5
12                         Angela: Queen of Hel Vol 1 1
13                  Captain America: Sam Wilson Vol 1 1
14                  Captain America: Sam Wilson Vol 1 2
15                       Captain America: White Vol 1 3
16                                    Civil War Vol 2 5
17                          Deadpool vs. Thanos 

In [556]:
%time agg = comic_plot_agg(s,'comic_corpus')

Wall time: 4min 23s


In [476]:
#agg

In [477]:
#%time preprocess_corpus_text(agg)

In [561]:
with open('comic_corpus.txt') as f:
    corpus = f.read()

In [562]:
len(corpus)

3461181

In [563]:
from gensim.models import Word2Vec

C:\Users\Rohit Suresh\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [565]:
%time corpus_tokens = preprocess_corpus_text(corpus)

Wall time: 15min 7s


In [584]:
%time embed = Word2Vec(corpus_tokens, size = 300, min_count = 3, sg = 1, iter=500, negative=10)

Wall time: 17min 49s


In [585]:
print(embed)

Word2Vec(vocab=7935, size=300, alpha=0.025)


In [586]:
embed_words = list(embed.wv.vocab)
print(embed_words[0:25])

['laura', 'allnew', 'wolverine', 'save', 'man', 'front', 'tower', 'assassinate', 'shot', 'head', 'fall', 'unconscious', 'remembers', 'conversation', 'logan', 'mission', 'fail', 'kill', 'drug', 'runner', 'attack', 'apologizes', 'explains', 'two', 'make']


In [626]:
print(embed.wv.__getitem__('thanos'))

[-6.84750974e-02  3.99098933e-01  5.17599918e-02 -7.15471685e-01
 -1.16666786e-01  2.57770121e-01  4.33020860e-01  1.36903417e+00
 -2.14118976e-03 -2.43924372e-02 -1.34073043e+00  1.77573133e-02
 -1.41533375e-01  7.59533839e-03 -7.30943918e-01 -2.62997281e-02
 -6.15421474e-01  5.05680330e-02 -4.43684489e-01  1.94002777e-01
 -5.40067494e-01  2.96885967e-01  1.76078603e-01  1.06869090e+00
  3.10422957e-01 -7.59450734e-01  2.90578574e-01 -2.09600553e-01
 -3.03681821e-01  7.62366116e-01 -1.72329336e-01 -1.97839892e+00
  2.10265428e-01  4.18375641e-01  5.12885273e-01 -2.76061535e-01
  4.35416639e-01  3.97232175e-01 -1.27180064e+00  6.20522320e-01
 -1.80213705e-01  8.07522833e-01  2.96129793e-01  7.22569525e-01
 -2.50769090e-02  2.39775211e-01 -5.38984299e-01  2.40873218e-01
 -4.76988256e-01  9.41541135e-01 -3.64971198e-02 -1.47116458e+00
  1.25569654e+00 -1.00636125e+00  1.32400370e+00  3.06404710e-01
 -6.53740108e-01  4.54577148e-01 -1.00918853e+00 -3.47118825e-01
  1.56850845e-01  5.22304

In [588]:
word_vectors = embed.wv

In [616]:
res = word_vectors.most_similar(positive=['wasp','hank'],negative=['janet'])
res

[('yellowjacket', 0.3419579565525055),
 ('ant', 0.2933281660079956),
 ('charlie27', 0.26632097363471985),
 ('handicap', 0.25764933228492737),
 ('sought', 0.2514813244342804),
 ('grat', 0.2513841688632965),
 ('lyander', 0.2510165274143219),
 ('piggy', 0.25082477927207947),
 ('weird', 0.2500923275947571),
 ('cahoot', 0.2467557191848755)]

In [617]:
word_vectors.similar_by_word('wasp')

[('yellowjacket', 0.658576488494873),
 ('ant', 0.4543679356575012),
 ('spectrum', 0.4529518187046051),
 ('prism', 0.39774370193481445),
 ('chairman', 0.3977375626564026),
 ('charlie27', 0.39174193143844604),
 ('scarlet', 0.38669300079345703),
 ('arctic', 0.37767261266708374),
 ('korvac', 0.36474430561065674),
 ('yondu', 0.3645530939102173)]

In [630]:
print(word_vectors.__getitem__('thanos'))

[-6.84750974e-02  3.99098933e-01  5.17599918e-02 -7.15471685e-01
 -1.16666786e-01  2.57770121e-01  4.33020860e-01  1.36903417e+00
 -2.14118976e-03 -2.43924372e-02 -1.34073043e+00  1.77573133e-02
 -1.41533375e-01  7.59533839e-03 -7.30943918e-01 -2.62997281e-02
 -6.15421474e-01  5.05680330e-02 -4.43684489e-01  1.94002777e-01
 -5.40067494e-01  2.96885967e-01  1.76078603e-01  1.06869090e+00
  3.10422957e-01 -7.59450734e-01  2.90578574e-01 -2.09600553e-01
 -3.03681821e-01  7.62366116e-01 -1.72329336e-01 -1.97839892e+00
  2.10265428e-01  4.18375641e-01  5.12885273e-01 -2.76061535e-01
  4.35416639e-01  3.97232175e-01 -1.27180064e+00  6.20522320e-01
 -1.80213705e-01  8.07522833e-01  2.96129793e-01  7.22569525e-01
 -2.50769090e-02  2.39775211e-01 -5.38984299e-01  2.40873218e-01
 -4.76988256e-01  9.41541135e-01 -3.64971198e-02 -1.47116458e+00
  1.25569654e+00 -1.00636125e+00  1.32400370e+00  3.06404710e-01
 -6.53740108e-01  4.54577148e-01 -1.00918853e+00 -3.47118825e-01
  1.56850845e-01  5.22304